# 训练U-Net++

## 配置pytorch环境

In [3]:
%cd /content/drive/MyDrive/U-Net++

/content/drive/MyDrive/U-Net++


In [ ]:
!pip list

In [ ]:
!pip install -r requirements.txt

## 导入包

In [6]:
import argparse
import os
from collections import OrderedDict
from glob import glob

import pandas as pd
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
# pip install PyYaml
import yaml
# https://github.com/albumentations-team/albumentations
# pip install -U albumentations
# python3.6+
from albumentations.augmentations.geometric.rotate import RandomRotate90
from albumentations.augmentations.geometric.resize import Resize 
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from tqdm import tqdm


import archs
import losses
from dataset import Dataset
from metrics import iou_score
from utils import AverageMeter, str2bool


In [7]:

ARCH_NAMES = archs.__all__
LOSS_NAMES = losses.__all__
LOSS_NAMES.append('BCEWithLogitsLoss')

"""

指定参数：
--dataset dsb2018_96 
--arch NestedUNet

"""

'\n\n指定参数：\n--dataset dsb2018_96 \n--arch NestedUNet\n\n'

In [8]:
def train(config, train_loader, model, criterion, optimizer):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    model.train()

    pbar = tqdm(total=len(train_loader))
    for input, target, _ in train_loader:
        #input = input.cuda()
        #target = target.cuda()

        # compute output
        if config.DEEP_SUPERVISION:
            outputs = model(input)
            loss = 0
            for output in outputs:
                loss += criterion(output, target)
            loss /= len(outputs)
            iou = iou_score(outputs[-1], target)
        else:
            output = model(input)
            loss = criterion(output, target)
            iou = iou_score(output, target)

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meters['loss'].update(loss.item(), input.size(0))
        avg_meters['iou'].update(iou, input.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])


In [9]:
def validate(config, val_loader, model, criterion):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        pbar = tqdm(total=len(val_loader))
        for input, target, _ in val_loader:
            # input = input.cuda()
            # target = target.cuda()

            # compute output
            if config.DEEP_SUPERVISION:
                outputs = model(input)
                loss = 0
                for output in outputs:
                    loss += criterion(output, target)
                loss /= len(outputs)
                iou = iou_score(outputs[-1], target)
            else:
                output = model(input)
                loss = criterion(output, target)
                iou = iou_score(output, target)

            avg_meters['loss'].update(loss.item(), input.size(0))
            avg_meters['iou'].update(iou, input.size(0))

            postfix = OrderedDict([
                ('loss', avg_meters['loss'].avg),
                ('iou', avg_meters['iou'].avg),
            ])
            pbar.set_postfix(postfix)
            pbar.update(1)
        pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])




In [10]:
class Config(object):
  NAME = "CT_Rock"
  EPOCHS = 50
  BATCH_SIZE = 8
  ARCH = 'NestedUNet'
  DEEP_SUPERVISION = False
  INPUT_CHANNELS = 3
  NUM_CLASSES = 1 
  INPUT_W = 512
  INPUT_H = 512
  LOSS = "BCEDiceLoss"
  DATASET = "final_rock_dataset"
  IMG_EXT = '.png'
  MASK_EXT = '.png'
  # choices=['Adam', 'SGD'],
  OPTIMIZER = 'Adam'
  # initial learning rate
  LR = 1e-3
  MOMENTUM = 0.9
  WEIGHT_DECAY = 1e-4
  NESTEROV = False
  # choices=['CosineAnnealingLR', 'ReduceLROnPlateau', 'MultiStepLR', 'ConstantLR'])
  SCHEDULER = 'CosineAnnealingLR'
  # minimum learning rate
  MIN_LR = 1e-5
  FACTOR = 0.1
  PATIENCE = 2
  MILESTONES = '1,2'
  GAMMA = 2/3
  # early stopping (default: -1)
  EARLY_STOPPING = -1
  NUM_WORKERS = 0

  def display(self):
    """Display Configuration values."""
    print("\nConfigurations:")
    for a in dir(self):
      if not a.startswith("__") and not callable(getattr(self, a)):
        print("{:30} {}".format(a, getattr(self, a)))
    print("\n")



In [11]:
config = Config()
config.display()


Configurations:
ARCH                           NestedUNet
BATCH_SIZE                     8
DATASET                        final_rock_dataset
DEEP_SUPERVISION               False
EARLY_STOPPING                 -1
EPOCHS                         50
FACTOR                         0.1
GAMMA                          0.6666666666666666
IMG_EXT                        .png
INPUT_CHANNELS                 3
INPUT_H                        512
INPUT_W                        512
LOSS                           BCEDiceLoss
LR                             0.001
MASK_EXT                       .png
MILESTONES                     1,2
MIN_LR                         1e-05
MOMENTUM                       0.9
NAME                           CT_Rock
NESTEROV                       False
NUM_CLASSES                    1
NUM_WORKERS                    0
OPTIMIZER                      Adam
PATIENCE                       2
SCHEDULER                      CosineAnnealingLR
WEIGHT_DECAY                   0.0001




## Train

In [12]:

if config.NAME is None:
    if config.DEEP_SUPERVISION:
        config.NAME = '%s_%s_wDS' % (config.DATASET, config.ARCH)
    else:
        config.NAME = '%s_%s_woDS' % (config.DATASET, config.ARCH)
os.makedirs('models/%s' % config.NAME, exist_ok=True)


# with open('models/%s/config.yml' % config.NAME, 'w') as f:
#     yaml.dump(config, f)

# define loss function (criterion)
if config.LOSS == 'BCEWithLogitsLoss':
    criterion = nn.BCEWithLogitsLoss().cuda()#WithLogits 就是先将输出结果经过sigmoid再交叉熵
else:
    criterion = losses.__dict__[config.LOSS]().cuda()

cudnn.benchmark = True

# create model
print("=> creating model %s" % config.ARCH)
model = archs.__dict__[config.ARCH](config.NUM_CLASSES,
                    config.INPUT_CHANNELS,
                    config.DEEP_SUPERVISION)
# ======================================================================================#
# model = model.cuda()  # 使用GPU

params = filter(lambda p: p.requires_grad, model.parameters())
if config.OPTIMIZER == 'Adam':
    optimizer = optim.Adam(
        params, lr=config.LR, weight_decay=config.WEIGHT_DECAY)
elif config.OPTIMIZER == 'SGD':
    optimizer = optim.SGD(params, lr=config.LR, momentum=config.MOMENTUM,
                          nesterov=config.NESTEROV, weight_decay=config.WEIGHT_DECAY)
else:
    raise NotImplementedError

if config.SCHEDULER == 'CosineAnnealingLR':
    scheduler = lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config.EPOCHS, eta_min=config.MIN_LR)
elif config.SCHEDULER == 'ReduceLROnPlateau':
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=config.FACTOR, patience=config.PATIENCE,
                                                verbose=1, min_lr=config.MIN_LR)
elif config.SCHEDULER == 'MultiStepLR':
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[int(e) for e in config.MILESTONES.split(',')], gamma=config.GAMMA)
elif config.SCHEDULER == 'ConstantLR':
    scheduler = None
else:
    raise NotImplementedError


=> creating model NestedUNet


获取数据集信息

In [13]:
# Data loading code
img_path = os.path.join('inputs', config.DATASET, 'images', '*' + config.IMG_EXT)
print(img_path)
img_ids = glob(img_path)
img_ids = [os.path.splitext(os.path.basename(p))[0] for p in img_ids]
print(img_ids)


inputs/final_rock_dataset/images/*.png
['3', '1', '2', '9', '7', '10', '4', '13', '11', '12', '8', '6', '5', '14', '15', '22', '23', '17', '20', '21', '18', '16', '19', '24', '31', '28', '27', '30', '29', '26', '25', '33', '35', '32', '34', '36', '45', '44', '37', '39', '42', '41', '43', '38', '40', '47', '46', '49', '48', '50']


划分训练集、验证集

In [14]:
train_img_ids, val_img_ids = train_test_split(img_ids, test_size=0.2, random_state=41)
print('train_dataset=========>\n',train_img_ids,'\n')
print('val_dataset=========>\n',val_img_ids,'\n')

train_dataset=========>
 ['47', '26', '10', '33', '39', '25', '32', '49', '40', '7', '38', '14', '21', '42', '30', '2', '41', '17', '37', '43', '45', '46', '6', '31', '29', '19', '18', '4', '48', '9', '27', '16', '24', '28', '1', '23', '34', '5', '36', '3'] 

val_dataset=========>
 ['50', '20', '12', '15', '35', '8', '11', '22', '44', '13'] 



### 数据增强

In [15]:
#数据增强：
train_transform = Compose([
    RandomRotate90(),
    transforms.Flip(),
    OneOf([
        transforms.HueSaturationValue(),
        transforms.RandomBrightness(),
        transforms.RandomContrast(),
    ], p=1),#按照归一化的概率选择执行哪一个
    Resize(config.INPUT_H, config.INPUT_W),
    transforms.Normalize(),
])

val_transform = Compose([
    Resize(config.INPUT_H, config.INPUT_W),
    transforms.Normalize(),
])


/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1746: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1772: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


In [16]:

train_dataset = Dataset(
    img_ids=train_img_ids,
    img_dir=os.path.join('inputs', config.DATASET, 'images'),
    mask_dir=os.path.join('inputs', config.DATASET, 'masks'),
    img_ext=config.IMG_EXT,
    mask_ext=config.MASK_EXT,
    num_classes=config.NUM_CLASSES,
    transform=train_transform)
val_dataset = Dataset(
    img_ids=val_img_ids,
    img_dir=os.path.join('inputs', config.DATASET, 'images'),
    mask_dir=os.path.join('inputs', config.DATASET, 'masks'),
    img_ext=config.IMG_EXT,
    mask_ext=config.IMG_EXT,
    num_classes=config.NUM_CLASSES,
    transform=val_transform)


In [17]:

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=True,  # 打乱
    num_workers=config.NUM_CLASSES,  # 数据集较小时，使用默认
    drop_last=True)#不能整除的batch是否就不要了
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    num_workers=config.NUM_CLASSES,
    drop_last=True)


In [ ]:

log = OrderedDict([
    ('epoch', []),
    ('lr', []),
    ('loss', []),
    ('iou', []),
    ('val_loss', []),
    ('val_iou', []),
])

best_iou = 0
trigger = 0
for epoch in range(config.EPOCHS):
    print('Epoch [%d/%d]' % (epoch, config.EPOCHS))

    # train for one epoch
    train_log = train(config, train_loader, model, criterion, optimizer)
    # evaluate on validation set
    val_log = validate(config, val_loader, model, criterion)

    if config.SCHEDULER == 'CosineAnnealingLR':
        scheduler.step()
    elif config.SCHEDULER == 'ReduceLROnPlateau':
        scheduler.step(val_log['loss'])

    print('loss %.4f - iou %.4f - val_loss %.4f - val_iou %.4f'
          % (train_log['loss'], train_log['iou'], val_log['loss'], val_log['iou']))

    log['epoch'].append(epoch)
    log['lr'].append(config.LR)
    log['loss'].append(train_log['loss'])
    log['iou'].append(train_log['iou'])
    log['val_loss'].append(val_log['loss'])
    log['val_iou'].append(val_log['iou'])

    pd.DataFrame(log).to_csv('models/%s/log.csv' %
                              config.NAME, index=False)

    trigger += 1

    if val_log['iou'] > best_iou:
        torch.save(model.state_dict(), 'models/%s/model.pth' %
                    config.NAME)
        best_iou = val_log['iou']
        print("=> saved best model")
        trigger = 0

    # early stopping
    if config.EARLY_STOPPING >= 0 and trigger >= config.EARLY_STOPPING:
        print("=> early stopping")
        break

    torch.cuda.empty_cache()


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [0/50]
